# Neural Nets from Scratch in Julia

## Lesson 7: Multiplication operation

* In this video we'll make add the forward and backward passes for multiplication
* [Documentation site here](https://mikesaint-antoine.github.io/SimpleGrad.jl)
* [Github repo here](https://github.com/mikesaint-antoine/SimpleGrad.jl)

In [1]:
## code so far

mutable struct Value{opType} <: Number
    data::Float64
    grad::Float64
    op::opType
end


struct Operation{FuncType, ArgTypes}
    op::FuncType
    args::ArgTypes
end

# constructor -- Value(data, grad, op)
Value(x::Number) = Value(Float64(x), 0.0, nothing);


import Base.show
function show(io::IO, value::Value)
    print(io, "Value(",value.data,")")
end


import Base.==
function ==(a::Value, b::Value)
     return a===b
end


import Base.+
function +(a::Value, b::Value)

    out = a.data + b.data    
    result = Value(out, 0.0, Operation(+, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



backprop!(val::Value{Nothing}) = nothing


function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(+), ArgTypes}
    
    # val = a + b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.grad
    val.op.args[2].grad += val.grad
    
end




function backward(a::Value)
    
    
    function build_topo(v::Value, visited=Value[], topo=Value[])
    
        if !(v in visited)
            
            push!(visited, v)
            
            if v.op != nothing
                for operand in v.op.args
                    
                    if operand isa Value
                        build_topo(operand, visited, topo)
                    end
                end 
            end
            
            push!(topo, v) 
            
            
        end
        return topo
    end
    
    
    
    topo = build_topo(a)
    
    a.grad = 1
    #da/da = 1
    
    for node in reverse(topo)
        backprop!(node)
    end
    
    
end


Base.promote_rule(::Type{<:Value}, ::Type{T}) where {T<:Number} = Value


In [2]:
import Base.*
function *(a::Value, b::Value)

    out = a.data * b.data    
    result = Value(out, 0.0, Operation(*, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end

* (generic function with 329 methods)

In [10]:
function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(*), ArgTypes}
    
    # val = a * b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.op.args[2].data * val.grad    
    val.op.args[2].grad += val.op.args[1].data * val.grad
    
end


# z = x * y


# dz/dx = y
# dz/dy = x

backprop! (generic function with 3 methods)

In [12]:
x = Value(2)
m = Value(4)
b = Value(7)


# y = m*x + b

y = b + m * x

println(y)

backward(y)

println(m.grad)
# dy/dm = x = 2

println(x.grad)
# dy/dx = m = 4

println(b.grad)
# dy/db = 1


Value(15.0)
2.0
4.0
1.0
